# Final Project Katherine Maki - BIOF 309

## Project Questions: How many bacterial features (at the genus level) are unique and overlapping across four major taxonomy databases? Are certain datasets more biased to include genera from a specific phylum?

### What is a bacterial genus?

Bacteria is classified taxonomically ranging from Phylum (high level) to species strain

![taxonomy](img/Taxonomy_copy2.png)

In microbiome analysis, 16S rRNA gene amplicon sequencing is a "culture independent" analysis technique used to classify bacteria based on hypervariable regions in the bacterial 16S rRNA gene. Bacteria is classified based on genetic sequences that are amplified from DNA extracted from a sample. For example, to noninvasively analyze the gut microbiome you can collect and extract DNA from fecal samples :)

## Four Databases that will be analyzed
#### Ribosomal Database Project
#### Greengenes
#### Silva 
#### Human Oral Microbiome Database

## Links to data
[Human Oral Microbiome Database](http://www.homd.org/?name=Download&taxonly=1)
[Silva Release 138](https://www.arb-silva.de/no_cache/download/archive/release_138/Exports/)
[Greengenes 13_8](https://docs.qiime2.org/2020.2/data-resources/)
[Ribosomal Database Project V16](https://mothur.org/wiki/rdp_reference_files/)

## Data Import

In [186]:
import pandas as pd
import numpy as np

## Since there are four datasets and they are all formatted differently, we will bring them in seperately and format them to extra characters are deleted, empty columns are changed to "NaN" and the genus data is subsetted

### Greengenes is imported first

In [187]:
#Import GreenGenes Dataset without header
gg = pd.read_table('data_input/gg_99_otu_taxonomy.txt', header=None, names=['id', 'taxonomy'])

In [188]:
#Split Single Taxonomy Column into Several Columns
gg = gg.taxonomy.str.split(";",expand=True)
gg.columns = ['Kingdom','Phylum','Class',
                     'Order','Family','Genus', 'Species']

In [189]:
#Get rid of the preceding type labels 
gg.Kingdom = gg.Kingdom.apply(lambda x: x.replace('k__',''))
gg.Phylum = gg.Phylum.apply(lambda x: x.replace('p__',''))
gg.Class = gg.Class.apply(lambda x: x.replace('c__',''))
gg.Order = gg.Order.apply(lambda x: x.replace('o__',''))
gg.Family = gg.Family.apply(lambda x: x.replace('f__',''))
gg.Genus = gg.Genus.apply(lambda x: x.replace('g__',''))
gg.Species = gg.Species.apply(lambda x: x.replace('s__',''))

In [190]:
#Replace blank spaces with "Not Classified" so all databases match
gg = gg.replace(r'^\s*$', "Not Classified", regex=True)

In [191]:
#Add column identifying the dataset
gg['Dataset'] = 'Greengenes'

In [194]:
#View output
gg.head()

,Kingdom,Phylum,Class,Order,Family,Genus,Species,Dataset
0,Bacteria,Cyanobacteria,Synechococcophycideae,Synechococcales,Synechococcaceae,Synechococcus,Not Classified,Greengenes
1,Bacteria,Proteobacteria,Alphaproteobacteria,Rickettsiales,Pelagibacteraceae,Not Classified,Not Classified,Greengenes
2,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Mycobacteriaceae,Mycobacterium,Not Classified,Greengenes
3,Bacteria,Firmicutes,Bacilli,Bacillales,Staphylococcaceae,Staphylococcus,Not Classified,Greengenes
4,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Anoxybacillus,kestanbolensis,Greengenes


In [195]:
#Pull out Phylum, Genus and Dataset Variables
gg_ss =  gg[['Phylum','Genus', 'Dataset']]

In [197]:
gg_ss.head()

,Phylum,Genus,Dataset
0,Cyanobacteria,Synechococcus,Greengenes
1,Proteobacteria,Not Classified,Greengenes
2,Actinobacteria,Mycobacterium,Greengenes
3,Firmicutes,Staphylococcus,Greengenes
4,Firmicutes,Anoxybacillus,Greengenes


### Next Silva is imported

In [198]:
#Import Silva Dataset without header
silva = pd.read_table('data_input/silva_taxonomy_7_levels.txt', header=None, names=['id', 'taxonomy'])

In [199]:
silva = silva.taxonomy.str.split(";",expand=True)
silva.columns = ['Kingdom','Phylum','Class',
                     'Order','Family','Genus', 'Species']

In [200]:
#Get rid of the preceding type labels 
silva.Kingdom = silva.Kingdom.apply(lambda x: x.replace('D_0__',''))
silva.Phylum = silva.Phylum.apply(lambda x: x.replace('D_1__',''))
silva.Class = silva.Class.apply(lambda x: x.replace('D_2__',''))
silva.Order = silva.Order.apply(lambda x: x.replace('D_3__',''))
silva.Family = silva.Family.apply(lambda x: x.replace('D_4__',''))
silva.Genus = silva.Genus.apply(lambda x: x.replace('D_5__',''))
silva.Species = silva.Species.apply(lambda x: x.replace('D_6__',''))

In [201]:
#Add column identifying the dataset
silva['Dataset'] = 'Silva'

In [202]:
#Only keep Bacteria in Silva Dataset as it also contains Archaea
silva_bac = silva[silva['Kingdom'] == 'Bacteria']

In [203]:
#Pull out Phylum, Genus and Dataset Variables
silva_ss =  silva_bac[['Phylum', 'Genus', 'Dataset']]

In [220]:
silva_ss.head()

,Phylum,Genus,Dataset
0,Proteobacteria,Klebsiella,Silva
1,Patescibacteria,Not Classified,Silva
2,Firmicutes,Tyzzerella 3,Silva
3,Proteobacteria,Not Classified,Silva
4,Proteobacteria,Pseudoalteromonas,Silva


In [ ]:
#Need to Change the Different Uncultured Bacteria Data in Silva to "Not Classified" to Match with GreenGenes
#Since there are multiple ways Silva writes uncultured (i.e. uncultured bacterium, uncultured organism) need 
    #search for uncultured and replace

In [221]:
#These versions didn't work 
#silva_ss.loc[silva_genus['Genus'].str.contains('uncultured')] = 'Not Classified'
#silva_ss['Genus'].str.lower().str.contains("uncultured") = "Not Classified"
#silva_ss["Genus"].str.contains("uncultured", case=False regex=False) = "Not Classified"
#silva_ss["Genus"].str.contains("uncultured", case=False regex=False) = "Not Classified"

In [218]:
#This works but gives me a warning message
silva_ss.loc[silva_ss['Genus'].str.contains('uncultured'), 'Genus'] = 'Not Classified'

/Users/katherineadurham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [219]:
silva_ss.head()

,Phylum,Genus,Dataset
0,Proteobacteria,Klebsiella,Silva
1,Patescibacteria,Not Classified,Silva
2,Firmicutes,Tyzzerella 3,Silva
3,Proteobacteria,Not Classified,Silva
4,Proteobacteria,Pseudoalteromonas,Silva


### Next Ribosomal Database Project is Imported

In [226]:
#Import RDP Dataset without header
rdp = pd.read_table('data_input/rdp_16s_v16_taxonomy.txt', header=None, names=['taxonomy'])

In [227]:
rdp.head()

,taxonomy
0,">AJ000684_S000004347;tax=d:Bacteria,p:""Actinob..."
1,">EF599163_S000871589;tax=d:Bacteria,p:""Proteob..."
2,">AY859683_S000631792;tax=d:Bacteria,p:""Actinob..."
3,">AY883036_S000481916;tax=d:Bacteria,p:""Proteob..."
4,">DQ656489_S000712413;tax=d:Bacteria,p:""Proteob..."


In [229]:
#Need to split twice with this dataset since there are two different seperators
#Semicolon between gene ID and taxonomy and 
rdp = rdp.taxonomy.str.split(";",expand=True)
rdp.columns = ['ID','taxonomy', 'blank']

AttributeError: 'DataFrame' object has no attribute 'taxonomy'

In [ ]:
silva = silva.taxonomy.str.split(";",expand=True)
silva.columns = ['ID','Kingdom','Phylum','Class',
                     'Order','Family','Genus', 'Species']

In [ ]:
silva.info()

In [99]:
gg_ss.describe()

,Genus,Dataset
count,93463,203452
unique,2062,1
top,Bacteroides,Greengenes
freq,2747,203452


In [ ]:
.shape

In [ ]:
Np.unique

In [ ]:
#Dropping duplicate names (one dataset)
vet_visits.drop_duplicates(subset="Genus")
#Dropping duplicate names (combined datasets)
unique_values= vet_visits.drop_duplicates(subset= ["Genus", "Dataset"])

Maybe I should keep phylum, because then I can count total unique genera and unique genera, straififed by per phylum

In [ ]:
#Unique genera by phylum sorted or unsorted
unique_values[["Phylum", "Dataset"]].value_counts()
unique_values[["Phylum", "Dataset"]].value_counts(sort=True)

In [ ]:
# Get the proportion of departments of each number and sort
dept_props_sorted = departments["department_num"].value_counts(sort=True, ascending=False, normalize=True)

## 3) Import your data
In the space below, import your data.
If your data span multiple files, read them all in.
If applicable, merge or append them as needed.

## 4) Show me the head of your data.

## 5) Show me the shape of your data

## 6) Show me the proportion of missing observations for each column of your data

In [124]:
#Filter out Archea and Eukaryotic Data

#Filter out not classified at genus level (i.e. "uncultured bacterium")
#Need to find the reverse of this
#is_not_uncultured = silva_genus["Genus"].isin(["uncultured bacterium", "uncultured organism", "uncultured"])
#df["uni_names"].str.lower().str.contains("berkeley") = "University of California, Berkeley"

NameError: name 'silva_genus' is not defined

## 7) Give me a problem statement.
Below, write a problem statement. Keep in mind that your task is to tease out relationships in your data and eventually build a predictive model. Your problem statement can be vague, but you should have a goal in mind. Your problem statement should be between one sentence and one paragraph.

## 8) What is your _y_-variable?
For final project, you will need to perform a statistical model. This means you will have to accurately predict some y-variable for some combination of x-variables. From your problem statement in part 7, what is that y-variable?